# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: José Luis Calvo Subirá

Url: https://colab.research.google.com/drive/1pKZPoUY5cYDaJgkjOGoDV6GDzQqUJ36t?usp=sharing

GitHub: https://github.com/jlcsub31/03MIAR---Algoritmos-de-Optimizacion

Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga <br>
>3. Combinar cifras y operaciones

**Descripción del problema:**

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

- Número de actores: 10
- Número de tomas : 30
- Actores/Tomas : https://bit.ly/36D8IuK
    - 1 indica que el actor participa en la toma
    - 0 en caso contrario

(*) La respuesta es obligatoria





                                        

In [333]:
#IMPORTS

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# EJECUTAR ANTES DE CONTINUAR !
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import random
import math
import numpy as np

In [57]:
tomas_vs_actores = np.array([
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
[0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
[1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
[1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
[1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
[1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
[1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
[1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
[1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
[1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
[1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
[1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
[1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
[1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

**(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?** <br>

Asumiendo que todas las tomas pueden doblarse en un día (no existe la restricción de 6 tomas/día), todos los actores podrían acudir el mismo día. En tal caso, el gasto sería independiente del orden en el que se realizan las tomas y participan los actores puesto que cobran por día. Ciertamente, podrían darse diversas combinaciones en cuanto al doblaje de tomas, pero el gasto es por día y no por toma, de modo que su valor final no se vería afectado por tales combianciones. En consecuencia, solo existiría una solución.

**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**

Si consideramos el caso en el que únicamente se pueden realizar 6 tomas/día, entonces el problema consiste en obtener las posibles combinaciones resultantes de elegir 6 tomas dentro de un conjunto de 30. Es importante destacar que no debe existir repetición y por supuesto el orden de elección sí que importa


$$ V^k_{n} = \frac{n}{(n-k)!} = \frac{30}{(30-6)!}$$

que resulta en más de 400 millones de combinaciones



## Modelo para el espacio de soluciones

**(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)**


Dado que el número total de tomas es de 30, y la restricción nos impone considerar un máximo de 6 tomas por día, asumiremos que el doblaje completo puede realizarse como mínimo en un total de 5 días. Además, teniendo en cuenta que cada actor cobra por día, es muy probable que la solución comprenda un total de 5 días. Sin embargo, con las restricciones impuestas, en este punto del modelado no podemos garantizar que exista una solución que reparta el doblaje de las tomas en 5 días. Si posteriormente se observa que no podemos alcanzar una solución, el modelo para el espacio de soluciones deberá revisarse.

Por ahora, considerando que la solución constará de 5 días, el modelo elegido consiste en una matriz 5x6, en el que cada fila representa un día, y cada columna representa la toma a doblar.

In [58]:
# FILA i -> día i
# COLUMNA j -> toma j

array = np.arange(1,31)
np.random.shuffle(array)

#Ejemplo de la estructura de una solucion
solucion_ejemplo = array.reshape((5,6))
print(solucion_ejemplo)

[[ 1 21  3 18 24  2]
 [20 11 19  6 12  4]
 [22 14  5 29 23 25]
 [ 8 10 15 13 26  9]
 [27 28 16 17  7 30]]


## Según el modelo para el espacio de soluciones

**(*)¿Cual es la función objetivo?**

**(*)¿Es un problema de maximización o minimización?**

La función objectivo consistirá en una función lineal que modelice el coste total de llevar a cabo el doblaje de todas las tomas. La única variable en este problema que actúa sobre el coste final es el salario de los actores (por día). Se debe alcanzar una solución que proporcione un coste mínimo (**minimización**).

Dicho coste dependerá del número de días que los actores deban acudir al estudio de doblaje.

In [180]:
def get_day_cost(day, data):
    '''
    Arguments:
        day: array con las tomas que van a doblarse en un dia
        data: matriz de datos originales del problema
    Returns:
        Coste total del dia, o lo que es lo mismo, numero de actores que han paricipado
        durante el dia
    '''
    #Day contiene el identificador de las tomas, se resta uno porque los arrays empiezan en
    #la posición 0
    mask = day - 1

    #Operaciones realizadas en este comando:
    # - data[mask,:]: obtiene las filas de la matriz original correspondientes a las
    #   tomas del dia.
    # - np.max(data[mask,:], axis=0): obtiene el maximo valor de cada columna de la matriz obtenida
    #   en el paso anterior. Si el actor ha participado en alguna toma del día, devolverá 1. Si no ha participado
    #   en ninguna, devolvera un cero.
    # - np.sum(np.max(data[mask,:], axis=0)): suma del array obtenido en el paso anterior. En otras palabras,
    #   número de actores partcipantes en las sesiones del día
    
    day_cost = np.sum(np.max(data[mask,:], axis=0))

    return day_cost

def get_solution_cost(solution, data):
    '''
    Arguments:
        solution: solución a evaluar
        data: matriz de datos originales del problema, que realciona las tomas con los
        actores
    Returns:
        Coste total de la solución de entrada
    '''
    total_cost = 0
    
    #Iteramos sobre cada día de la solución de entrada
    for day in solution:
        day_cost = get_day_cost(day, data)
        
        #Coste acumulado
        total_cost += day_cost 
    return total_cost

In [181]:
# Coste de la solución de ejemplo
x = get_solution_cost(solucion_ejemplo, tomas_vs_actores)

print(x)

38


En el [anexo 1](#anexo1) propongo una implementación más detallada y exponiendo razonadamente en el código cuál es la finalidad de cada línea. También se presenta un ejemplo, sacando logs en consola, de la obtención del coste de la solución de ejemplo.

## Diseña un algoritmo para resolver el problema por fuerza bruta

**WARNING**: Cuidado al ejecutar

In [90]:
# Se usa itertools para calcular las posibles soluciones
def variations(n, k):
    '''
    Arguments:
        n: tamaño total del conjunto de elementos
        k: tamaño de los subconjuntos
    Returns:
        Lista de las posibles soluciones
    '''
    return list(itertools.permutations(np.arange(1,n+1), k))

def brute_force(data, m = 6):
    '''
    Arguments:
        data: matriz de datos originales del problema, que relaciona las tomas con los
        actores
        m: tamaño de los subconjuntos
    Returns:
        Solución de coste menor
        Valor del coste total
    '''
    min_cost = float('inf')
    min_cost_sol = []

    n = len(data)
    restriction = m
    all_solutions = variations(n, restriction)

    for sol in all_solutions:
        sol_cost = get_solution_cost(sol, data)
        if sol_cost < min_cost:
            min_cost_sol = sol
            min_cost = sol_cost
    return min_cost_sol, min_cost

#sol_brute_force, cost_brute_force = brute_force(tomas_vs_actores)

# print(sol_brute_force)
# print(cost_brute_force)

**Calcula la complejidad del algoritmo por fuerza bruta**

La función 'permutations' de itertools genera todas las soluciones existentes, recordando de apartados anteriores, este espacio esta definido por

$$ V^k_{n} = \frac{n}{(n-k)!} $$

De este modo la complejidad de este algoritmo basado en la fuerza bruta es de $O(n!)$

### (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Debido a la complejidad del problema que nos ocupa, usar algoritmos deterministas que nos aseguren la solución óptima queda descartado. Los algoritmo heurísticos ofrecen buenas soluciones en problemas en los que los de tipo determinista no son viables, aunque no hay forma de conocer el grado de aproximación a la solución óptima. 

Un aspecto a tener en cuenta a la hora de decidirse por una técnica metaheurística es el riesgo de quedar atrapados en óptmios locales (en este caso, mínimos). Es por ello que el diseño del agoritmo debe presentar un equilibrio entre intensificación y diversificación, es decir, centrarse en zonas consideredas a la vez que explora regiones del espacio de soluciones desconocidas. Una forma de escapar de los mínimos locales es aceptar en ocasiones soluciones peores. La técnica de "simulated annealing" introduce una probabilidad de aceptación de este tipo de soluciones.

La técnica de "recocido simulado" se basa en los procesos metalúrgicos en los que un material es calentado hasta una elevada temperatura y enfriado posteriormente. A altas temperaturas, los átomos pueden moverse impredeciblemente, a menudo elminando impurezas a medida que el material se enfría en un cristal puro.

De forma similar, el algoritmo a diseñar comenzará con una elevada temperatura T, y se ira reduciendo de tal forma que cuando su valor descienda de una determinada cantidad, el algoritmo se detendrá (criterio de parada). La función de la temperatura T no consiste únicamente en actuar como cirterio de parada, ya que la probabilidad de aceptación mencionada anteriormente también depende de ella sgún la siguiente fórmula:

$$ p = e^{\frac{-d}{T}} $$

donde $d$ es la diferencia del valor de la función objetivo evaluada en dos soluciones distintas.

Es importante destacar, que cuanto mayor sea el valor de T, la probabilidad de aceptación de una mala solución crece permitiendo diversificar en el espacio de soluciones, mientras que a medidad que T reduce su valor, dicha probabilidad decrece y por lo tanto estaremos intensificando la búsqueda a partir de una buena solución.

Los conceptos teóricos anteriores se traducen en el siguiente pseudocódigo:

    T ← Initialise T <br/>
    S_ref ← Initialise S_ref <br/>
    S_opt ← Initialise S_opt <br/>
        while T > k: <br/>
            S_neighbor ← a randomly selected successor of S
            if f(S_neighbor) < f(S_opt):
                S_opt ← S_neighbor
            random r(0,1)
            Δf ← |f(S_neighbor) - f(S_ref)|
            if f(S_neighbor) < f(S_ref) or r > exp(-Δf/T)
                S_ref ← S_neighbor
            T ← decrease_T(t)
        end while
    return S_opt
    
- Decenso de temperatura: **decrease_T**
- Probabilidad de aceptación: **is_solution_accepted**
- Inicialización de S_ref: **generate_random_solution**, **generate_random_solution_search**
- Cálculo de S_neighbor: **generate_neighbor**

El diseño de la función de generación de soluciones vecinas es clave en este algoritmo. Una solución tiene forma de matriz en la que las filas representan los días, y las columnas las tomas para ese día. El problema no impone restricciones sobre el orden de las tomas en un mismo día, de modo que intercambiar valores únicamente a lo largo de las filas de matriz, no produce ningún cambio al evaluar las soluciones con la función de evaluación (la función de coste). Lo que nos interesa es realizar cambios a lo largo de las columnas. Por lo tanto, se ha optado por realizar intercambios aleatorios a lo largo de las columans de acuerdo a los siguientes puntos:
- Se obtiene aleatoriamente el número de columnas a cambiar
- Se obtienen aleatoriamente las columnas sobre las que actuar
- Se realizan intercambios aleatorias en cada una de las columnas afectadas

Ejemplo:

    [[ 1 21  3 18 24  2]
     [20 11 19  6 12  4]
     [22 14  5 29 23 25]
     [ 8 10 15 13 26  9]
     [27 28 16 17  7 30]]
     
    - Número de columnas a cambiar: 4
    - columnas sobre las que actuar: [4 2 5 3], columnas 0 y 1 no se alterarán
    - Intercambios en las columnas seleccionadas, dando como resultado:
    
    [[ 1 21 19 29 24 25]
     [20 11  3  6 12  4]
     [22 14 16 17 23  2]
     [ 8 10 15 13  7 30]
     [27 28  5 18 26  9]]
    

A continuación se definen las funciones necesarias para el diseño del algortimo. En el [anexo 1](#anexo1) se presenta un ejemplo de algunas de ellas proporcionando 'logs' en consola.

In [337]:
def decrease_T(T):
    '''
    Reduce el valor de T
    Arguments:
        T: valor de la temperatura
    Returns:
        Valor de la temperatura reducido en un 1%
    '''
    return T*0.99

def is_solution_accepted(T, d):
    '''
    Probabilidad de aceptación de una solución
    Arguments:
        T: valor de la temperatura
        d: diferencia entre el valor de las dos soluciones
    Returns:
        Valor booleano que representa aceptación o rechazo de una peor solución
    '''
    if random.random() <  math.exp( -1*d / T)  :
        return True
    else:
        return False

def generate_random_solution():
    '''
    Arguments: void
    Returns:
        Solución aleatoria
    '''
    array = np.arange(1,31)
    np.random.shuffle(array)

    return array.reshape((5,6))

def generate_neighbor(ref, data, n=10, verbose=False):
    '''
    Desc: Vecindario de una solución, se obtienen n soluciones vecinas y se devuelve
    la mejor de a cuerdo a la función de evaluación
    Arguments:
        ref: Solución a partir de la cual serán generadas soluciones vecinas
        data: matriz de datos originales del problema, que relaciona las tomas con los
        actores 
        n: número de soluciones vecinas a generar
        verbose: escribir logs por pantalla
    Returns:
        La mejor solución vecina según la función de evaluación, es decir, la función
        objetivo que calcula el coste
    '''
    
    neighbor_min_cost = []
    min_cost = float('inf')
    if(verbose): print(ref)
    
    #n generaciones
    for it in np.arange(n):
        if(verbose): print(f'Vecino número: {it + 1}')
        
        neighbor = ref.copy()
        
        #Se eligen al azar n columnas para mezclar, con un mínimo de 2
        number_of_columns_to_permute = random.randint(2,ref.shape[1]-1)
        #Se eligen al azar las n columnas para mezclar
        colmuns_to_permute = np.random.choice(ref.shape[1], number_of_columns_to_permute, replace=False)

        if(verbose):
            print('Columnas a mezclar')
            print(colmuns_to_permute)
            
        for i in colmuns_to_permute:
            copy_column = neighbor[:,i]
            np.random.shuffle(copy_column)
            neighbor[:,i] = copy_column
        
        #Se calcula el coste de la solución vecina
        cost_neighbor = get_solution_cost(neighbor, data)
        
        if(verbose):
            print('Solución vecina')
            print(neighbor)
            print('Coste de la solución vecina')
            print(cost_neighbor)
            print('---------------------------')

        #Si la solución vecina en consideración tiene coste menor que la mejor
        #solución vecina encontrada hasta el momento, la primera pasa a ser la
        #mejor
        if cost_neighbor < min_cost:
            min_cost = cost_neighbor
            neighbor_min_cost = neighbor
            
    if(verbose):
        print('***************************')
        print('Mejor solución vecina:')
        print(neighbor_min_cost)
        print('Coste de la mejor solución vecina:')
        print(min_cost)
        print('***************************')
        
    return neighbor_min_cost, min_cost
    

Al comienzo del algoritmo, debemos inicializar una posible solución. Una posibilidad es elegir una al azar. Otra más recomendable es elegir una solución inicial que haya sido obtenida a través de otra técnica. En este caso, la función **simulated_annealing** aceptará un valor "flag" para activar/desactivar la inicialización de la solución mediante una búsqueda aleatoria rápida de acuerdo a la siguiente función:

In [334]:
def generate_random_solution_search(data = tomas_vs_actores, n = 10):
    '''
    Búsqueda aleatoria de una solución.
    Arguments:
        data: matriz de datos originales del problema, que relaciona las tomas con los
        actores
        n: número de iteraciones. Criterio de parada
    Returns: mejor solución obtenida aleatoriamente
    '''
    min_cost = float('inf')
    random_solution_selected = []
    
    for i in np.arange(n):
        array = np.arange(1,31)
        np.random.shuffle(array)
        random_solution = array.reshape((5,6))
        cost_random_solution = get_solution_cost(random_solution, data)
        
        if cost_random_solution < min_cost:
            min_cost = cost_random_solution
            random_solution_selected = random_solution
            
    return random_solution_selected, min_cost
            

In [335]:
def simulated_annealing(data, T, initialise_random_search=False):
    '''
    Alcanza una solución del problema de entrada. No tiene por qué devolver el óptimo
    global, lo más probable es que se trate de un óptimo local.
    Arguments:
        data: matriz de datos originales del problema, que relaciona las tomas con los
        actores
        T: temperatura inicial
        initialise_random_search:
            True: la solución inicial se calcula con búsqueda aleatoria
            False: se genera una solución aleatoria como solución inicial
    Returns:
        Solución óptima encontrada.
    '''
    print("Starting simulated_annealing...")
    reference_solution = []
    reference_solution_cost = 0
    
    if(initialise_random_search):
        reference_solution, reference_solution_cost = generate_random_solution_search(data)
    else:
        reference_solution = generate_random_solution()
        reference_solution_cost = get_solution_cost(reference_solution, data)
    
    print("Solución incial:")
    display(reference_solution)
    print(f"Con un coste de: {reference_solution_cost}")
    print('----------------')
  
    optimal_solution = []
    optimal_solution_cost = float('inf')
  
    iterator =0
    print("Starting loops...")
    while T > .0001:
        iterator += 1
        neighbor_solution, neighbor_solution_cost = generate_neighbor(reference_solution, data, n=50)

        # Si es la mejor solución de todas se guarda
        if neighbor_solution_cost < optimal_solution_cost:
            optimal_solution = neighbor_solution
            optimal_solution_cost = neighbor_solution_cost

        if neighbor_solution_cost < reference_solution_cost or is_solution_accepted(T, abs(reference_solution_cost - neighbor_solution_cost)):
            reference_solution = neighbor_solution
            reference_solution_cost = neighbor_solution_cost

        #Se disminuye la temperatura
        T = decrease_T(T)

    print(iterator)
    return optimal_solution, optimal_solution_cost

### **(*)Calcula la complejidad del algoritmo**

- Bucle while: con una reducción de un 1% de T en cada iteración, el número de iteraciones n vendrá dado por:
$$0.99^{n}*T > 0.0001$$

De esta forma n se calcularía como:

$$ n = log_{0.99}(0.0001/T)$$ (1)

Para T = 10000,

$$ n = 1832 iteraciones $$

El número total será entonces 1833 ya que $0.99^{1832}*10000 = 0.0001008 > 0.0001$ por lo que se realiza una iteración más. El número de iteraciones depende por tanto de la temperatura según (1). Si hacemos un cambio de base para obtener la expresión logarítmica en base decimal (según la propiedad 1 del [anexo 2](#anexo2)), el número de iteraciones podrá expresarse como:

$$ n = log_{0.99}(0.0001/T = log(0.0001/T) * log_{0.99}(10) = log(0.0001/T) * -229.105 $$ (2)

Aplicando ahora la propiedad 2 de del [anexo 2](#anexo2), n será igual a:

$$ n = 229.105*log(T) - 229.105*log(0.0001) $$

- Operaciones en el bucle while:

    La operación predominante es la generación de soluciones vecinas:

    - se realizan m iteraciones, siendo m el número de soluciones vecinas que se quieren generar
    - En ducho bucle, predomina la función get_solution_cost
        - Se realizan un total de 5 iteraciones de get_day_cost (una por cada día), procedente de 30 tomas divido para 6 tomas/día
            - Numpy max -> $O(n)$, siendo n el número de actores
            - Numpy max -> $O(n)$, siendo n el número de actores
            - Slicing de tamaño k = 6 (6 tomas por día) -> $O(6)$ de acuerdo con https://wiki.python.org/moin/TimeComplexity
        
    Complejidad en función de los datos de entrada: $O(n_{vecinas}*n_{días}/n_{tomas/día}*(n_{actores}+n_{tomas}))$
    
Complejidad: $$O(n_{vecinas}*n_{días}/n_{tomas/día}*(n_{actores}+n_{tomas}))*(229.105*log(T) - 229.105*log(0.0001))$$


### **Ejecución del algoritmo**

- Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

- Aplica el algoritmo al juego de datos generado

In [330]:
#Ejecución del algoritmo SIMULATED ANNEALING
T = 10000
sol_simulated_annealing, cost_simulated_annealing = simulated_annealing(tomas_vs_actores, T, initialise_random_search=True)

print("Solución óptima alcanzada:")
display(sol_simulated_annealing)
print(f"Con un coste de: {cost_simulated_annealing}")

Starting simulated_annealing...
Solución incial:


array([[13, 19,  5,  2, 30, 27],
       [ 9,  3, 21, 29, 23,  4],
       [24,  1, 28, 26, 12, 15],
       [11, 10, 14, 17,  8, 18],
       [25, 16, 22,  6, 20,  7]])

Con un coste de: 35
----------------
1833
Solución óptima alcanzada:


array([[11, 10, 21, 26,  8, 15],
       [25, 16, 22,  2, 30, 27],
       [ 9,  1, 28, 29, 12,  7],
       [13,  3,  5,  6, 20,  4],
       [24, 19, 14, 17, 23, 18]])

Con un coste de: 30


### **Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**

Las únicas referencias usadas en la realización de este trabajo han sido las diapositivas de clase, así como el manual de la asignatura y lo aprendido en las activadas guiadas.

### **Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**

Existen muchas posibilidades de ampliar y combinar las restricciones del problema dado. Por ejemplo, sería interesante considerar la posibilidad de realizar "horas extra". Es decir, se le podría pedir a un actor realizar una toma más por día a cambio de un suplemento en el salario (lo cual afectaría a nuestra función objetivo). No obstante, cabría la posibilidad de que añadir una secuencia más sobre el salario de un actor permitiera que en otro día de la semana, dos de ellos no tuviesen la obligación de venir, por ejemplo, reduciendo así el coste final.

Otra restricción añadida podría ser la imposición de un orden de doblaje de las secuencias. Dadas las características del problema actual, no es necesario tener en consideración el orden de las secuencias, ya sea en el mismo día o en toda la semana (los 5 días)

Cualquier restricción adicional que suponga un incremento del espacio de soluciones, complicaría la resolución del problema a través del método propuesto (simulated annealing). Es posible que tuviéramos que recurrir a técnicas poblacionales como los algoritmos genéticos para asegurarnos de que las soluciones descendientes en cada iteración mejoran la población. Además, las restricciones podrían añadirse al problema fácilmente como penalizaciones en la función de evaluación.

# ANEXO 1<a name="anexo1">

In [249]:
#Solución de ejemplo para el apéndice
array = np.arange(1,31)
np.random.shuffle(array)

#Ejemplo de la estructura de una solucion
solucion_ejemplo_apendix = array.reshape((5,6))
print(solucion_ejemplo_apendix)

[[20 14  9  8 25 12]
 [10  5  1 23 18  7]
 [ 6 22 24 17 21 30]
 [29 16 15  3 28 26]
 [13 19 27  4  2 11]]


In [290]:
# Dada la solución de ejemplo y la matriz de datos del problema
def get_day_cost_B(day, data):
    '''
    Arguments:
        day: array con las tomas que van a doblarse en un dia
        data: matriz de datos originales del problema
    Returns:
        Coste total del dia, o lo que es lo mismo, numero de actores que han paricipado
        durante el dia
    '''
    #Day contiene el identificador de las tomas, se resta uno porque los arrays empiezan en
    #la posición 0
    mask = day - 1
    print('Secuencias en el día')
    print(day)
    print('Mask')
    print(mask)
    
    #Filas de las tomas para doblar el día de entrada. Las columnas no se filtran, ya que contienen
    #la información sobre los actores
    tomas_dia_vs_actores = data[mask,:]
    print('Tomas del día relacionadas con los actores')
    print(tomas_dia_vs_actores)
    
    #1 para los que han participado en alguna toma, 0 para los que no
    #Al usar la función "max", solo se contabiliza a cada actor una vez si procede
    actores_participantes_si_no = np.max(data[mask,:], axis=0)
    print('Partición de los actores')
    print(actores_participantes_si_no)
    
    day_cost = np.sum(actores_participantes_si_no)

    print(day_cost)
    print('-----------------------------')
    return day_cost


def get_solution_cost_B(solution, data):
    '''
    Arguments:
        solution: solución a evaluar
        data: matriz de datos originales del problema, que relaciona las tomas con los
        actores
    Returns:
        Coste total de la solución de entrada
    '''
    total_cost = 0
    for day in solution:
        day_cost = get_day_cost_B(day, data)
        total_cost += day_cost 
    return total_cost

x = get_solution_cost_B(solucion_ejemplo_apendix, tomas_vs_actores)
print(f"COSTE TOTAL: {x}")

Secuencias en el día
[20 14  9  8 25 12]
Mask
[19 13  8  7 24 11]
Tomas del día relacionadas con los actores
[[1 0 1 1 1 0 0 0 0 0]
 [1 0 1 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1]
 [1 1 1 1 0 1 0 0 0 0]]
Partición de los actores
[1 1 1 1 1 1 0 0 0 1]
7
-----------------------------
Secuencias en el día
[10  5  1 23 18  7]
Mask
[ 9  4  0 22 17  6]
Tomas del día relacionadas con los actores
[[1 1 0 0 0 1 0 0 1 0]
 [0 1 0 1 0 0 0 1 0 0]
 [1 1 1 1 1 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 1 0 1 1 0 0 0 0 0]]
Partición de los actores
[1 1 1 1 1 1 0 1 1 0]
8
-----------------------------
Secuencias en el día
[ 6 22 24 17 21 30]
Mask
[ 5 21 23 16 20 29]
Tomas del día relacionadas con los actores
[[1 1 0 1 1 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0]]
Partición de los actores
[1 1 1 1 1 1 0 1 0 0]
7
-----------------------------
Secuencias en el dí

In [291]:
# TEST de la función de generate_neighbor

# Cinco generaciones (n=5), verbose = True para sacar los resultados por pantalla
sol_test_neighbor, cost_test_neighbor = generate_neighbor(solucion_ejemplo, tomas_vs_actores, n=5, verbose = True)

[[ 1 21  3 18 24  2]
 [20 11 19  6 12  4]
 [22 14  5 29 23 25]
 [ 8 10 15 13 26  9]
 [27 28 16 17  7 30]]
Vecino número: 1
Columnas a mezclar
[0 1 5 3 2 4]
Solución vecina
[[22 28 15  6 23  2]
 [ 8 10  5 13  7 25]
 [ 1 14  3 17 24  4]
 [20 11 19 29 26 30]
 [27 21 16 18 12  9]]
Coste de la solución vecina
38
---------------------------
Vecino número: 2
Columnas a mezclar
[2 0]
Solución vecina
[[20 21  3 18 24  2]
 [ 8 11 19  6 12  4]
 [ 1 14 16 29 23 25]
 [27 10  5 13 26  9]
 [22 28 15 17  7 30]]
Coste de la solución vecina
37
---------------------------
Vecino número: 3
Columnas a mezclar
[4 2 5 3]
Solución vecina
[[ 1 21 19 29 24 25]
 [20 11  3  6 12  4]
 [22 14 16 17 23  2]
 [ 8 10 15 13  7 30]
 [27 28  5 18 26  9]]
Coste de la solución vecina
38
---------------------------
Vecino número: 4
Columnas a mezclar
[4 5 3 2]
Solución vecina
[[ 1 21 15 17  7 25]
 [20 11 19 13 24  4]
 [22 14 16  6 23 30]
 [ 8 10  3 18 26  2]
 [27 28  5 29 12  9]]
Coste de la solución vecina
39
--------------

# ANEXO 2<a name="anexo2">

PROPIEDAD 1: CAMBIO DE BASE

$$ log_{b}(x) = \frac{log_{c}(x)}{log_{c}(b)} $$

PROPIEDAD 2

$$ log(\frac{A}{B}) = log(A) - log(B) $$

